# Import dependencies

In [5]:
import pandas as pd       
import numpy as np    
import requests           
import geopandas as gpd    
import shapely
from shapely.geometry import LineString  

print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"requests version: {requests.__version__}")
print(f"geopandas version: {gpd.__version__}")
print(f"shapely version: {shapely.__version__}")

pandas version: 1.5.3
numpy version: 1.23.5
requests version: 2.29.0
geopandas version: 0.13.2
shapely version: 2.0.4


# Download roads

In [ ]:
# Define the Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Define the bounding box for Africa (approximately)
# Coordinates are (min_latitude, min_longitude, max_latitude, max_longitude)
bbox = (-35.0, -26.0, 38.0, 59.0)

# Function to create a grid of bounding boxes
def create_bbox_grid(min_lat, min_lon, max_lat, max_lon, n):
    lat_points = np.linspace(min_lat, max_lat, n+1)
    lon_points = np.linspace(min_lon, max_lon, n+1)
    bboxes = []
    for i in range(n):
        for j in range(n):
            bboxes.append((lat_points[i], lon_points[j], lat_points[i+1], lon_points[j+1]))
    return bboxes

# Define the number of divisions per side
divisions = 2
bboxes = create_bbox_grid(bbox[0], bbox[1], bbox[2], bbox[3], divisions)


# Function to fetch data for a given bounding box
def fetch_bbox_data(bbox):
    print(bbox)
    overpass_query = f"""
    [out:json][timeout:3600];
    (
      way["highway"~"motorway|trunk|primary|secondary|tertiary"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      node(w);
    );
    out body;
    >;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()
    nodes = {element['id']: (element['lon'], element['lat']) for element in data['elements'] if element['type'] == 'node'}
    
    def extract_attributes(element):
        tags = element.get('tags', {})
        node_ids = element.get('nodes', [])
        coords = [nodes[node_id] for node_id in node_ids if node_id in nodes]
        return {
            'osmid': element['id'],
            'highway': tags.get('highway'),
            'surface': tags.get('surface'),
            'width': tags.get('width'),
            'lanes': tags.get('lanes'),
            'oneway': tags.get('oneway', 'no') == 'yes',
            'geometry': LineString(coords) if coords else None
        }

    ways = [extract_attributes(element) for element in data['elements'] if element['type'] == 'way']
    gdf_edges = gpd.GeoDataFrame([way for way in ways if way['geometry'] is not None])
    gdf_edges.set_crs(epsg=4326, inplace=True)
    
    return gdf_edges

# Fetch data for all bounding boxes and combine
gdfs = [fetch_bbox_data(bbox) for bbox in bboxes]
gdf_combined = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Check the size of the resulting GeoDataFrame
print(f"Number of rows: {gdf_combined.shape[0]}")

# Print detailed info about the GeoDataFrame
print(gdf_combined.info())

output_filename = "data/Africa_roads_raw.shp"
# Save the GeoDataFrame to a shapefile
gdf_combined.to_file(output_filename)